In [34]:
grid = []

with open("input.txt") as file:
    for line in file:
        grid.append(line.strip())

grid

['.....................#................#...#.....#.................................................#...........#...................',
 '.............#....#.#.................................#....................................................#.........#............',
 '...........................................#...................................................................#......#...........',
 '....................................................#................#.......#.............#......................................',
 '..........................................#...##.#....................................#....#.........#............................',
 '....................................#.............................................#.......#................#.......###............',
 '..##.......#......................#..#........................................................#...................#...............',
 '.....#...........#..................................#

In [35]:
pos = ()

dir = (-1, 0)

for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == "^":
            pos = (i, j)

pos

(43, 37)

In [36]:

done = False
steps = 1 # start at 1 for starting position

while not done:


    grid[pos[0]] = grid[pos[0]][:pos[1]] + "X" + grid[pos[0]][pos[1] + 1:]
    new_pos = (pos[0] + dir[0], pos[1] + dir[1])

    if new_pos[0] < 0 or new_pos[0] >= len(grid) or new_pos[1] < 0 or new_pos[1] >= len(grid[new_pos[0]]):
        done = True
        break

    if grid[new_pos[0]][new_pos[1]] == "#":
       dir = (dir[1], -dir[0]) # turn 90 degrees cw

    if grid[new_pos[0]][new_pos[1]] == "." or grid[new_pos[0]][new_pos[1]] == "^" or grid[new_pos[0]][new_pos[1]] == "X":
        if grid[new_pos[0]][new_pos[1]] != "X" and grid[new_pos[0]][new_pos[1]] != "^":
            steps += 1

        pos = new_pos

steps

5318

In [37]:
grid

['.....................#................#...#.....#.................................................#...........#...................',
 '.............#....#.#.................................#....................................................#.........#............',
 '...........................................#..........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#......#...........',
 '....................................................#.X..............#.......#.............#..................X...................',
 '..........................................#...##.#....X...............................#....#.........#........X...................',
 '....................................#..........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.......#................#..X....###............',
 '..##.......#......................#..#.........X......X..........................X............#...............X...#...............',
 '.....#...........#.............................X....#

In [6]:

def mark_backwards(pos, dir, grid):
    tmp_pos = pos
    tmp_dir = dir
    while True:
        new_pos = (tmp_pos[0] - tmp_dir[0], tmp_pos[1] - tmp_dir[1])

        if in_grid(new_pos, grid):
            break
        elif grid_blocked(new_pos, grid):
            tmp_dir = (-tmp_dir[1], tmp_dir[0]) # turn 90 degrees ccw
        elif grid[new_pos[0]][new_pos[1]] & dir_map[tmp_dir]:
            break # already marked with this direction, so we can reach this field, we dont need to continue
        else: 
            state = dir_map[tmp_dir] | grid[new_pos[0]][new_pos[1]]
            grid[new_pos[0]] = grid[new_pos[0]][:new_pos[1]] + str(state) + grid[new_pos[0]][new_pos[1] + 1:] # Write direction to grid
            tmp_pos = new_pos

    return grid

def check_loop(pos, dir, grid):
    # check if we can turn to create loop, so if the field is marked with the turn direction than we can crate loop
    # checking by combining the binaries, if one bit is set we will get a True
    turn_dir = (-dir[1], dir[0])
    return bool(int(grid[pos[0]][pos[1]]) & dir_map[turn_dir])

def grid_free(pos, grid):
    return grid[pos[0]][pos[1]] == "." or grid[pos[0]][pos[1]] == "S"

def grid_walked(pos, grid):
    return grid[pos[0]][pos[1]].isdigit()

def grid_blocked(pos, grid):
    return grid[pos[0]][pos[1]] == "#"

def in_grid(pos, grid):
    return pos[0] >= 0 and pos[0] < len(grid) and pos[1] >= 0 and pos[1] < len(grid[pos[0]])

grid = []
pos = ()
dir = (-1, 0)

dir_map = {
    (-1, 0) : 0b0001,
    (0, 1) : 0b0010,
    (1, 0) : 0b0100,
    (0, -1) : 0b1000,
}


with open("input.txt") as file:
    for line in file:
        grid.append(line.strip())


for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == "^":
            pos = (i, j)
            break

grid[pos[0]] = grid[pos[0]][:pos[1]] + 'S' + grid[pos[0]][pos[1] + 1:] # Write empty space for algo to work :S

done = False
steps = 1 # start at 1 for starting position

grid = mark_backwards(pos, dir, grid)

ans = 0
while not done:

    new_pos = (pos[0] + dir[0], pos[1] + dir[1])

    if not in_grid(new_pos, grid):
        done = True
        break

    if grid_blocked(new_pos, grid):
        # turn 90 degrees cw
        dir = (dir[1], -dir[0]) # turn 90 degrees cw
        # grid[pos[0]] = grid[pos[0]][:pos[1]] + str(dir_map[dir]) + grid[pos[0]][pos[1] + 1:]

        # mark everything behind for loop detection
        grid = mark_backwards(pos, dir, grid)

    elif grid_free(new_pos, grid):
        # walk and mark position with direction
        pos = new_pos
        if grid[pos[0]][pos[1]] != "S": # keep starting position marked
            grid[pos[0]] = grid[pos[0]][:pos[1]] + str(dir_map[dir]) + grid[pos[0]][pos[1] + 1:] # Write direction to grid

    elif grid_walked(new_pos, grid):
        # already been here, check for loop
        if check_loop(new_pos, dir, grid):
            print("Loop detected")
            for line in grid:
                if grid.index(line) == 1:
                    print(grid[pos[0]][:pos[1]] + "O" + grid[pos[0]][pos[1] + 1:])
                else:
                    print(line)
            print('')

            ans += 1

        pos = new_pos    
        state = dir_map[dir] | int(grid[pos[0]][pos[1]]) # combine directions
        grid[pos[0]] = grid[pos[0]][:pos[1]] + str(state) + grid[pos[0]][pos[1] + 1:]

        pos = new_pos


ans

Loop detected
.....................#................#...#.....#.................................................#...........#...................
..#.......#.................12222222O1..............................4..................#...#.....................#................
...........................................#...................................................................#......#...........
....................................................#................#.......#.............#......................................
..........................................#...##.#....................................#....#.........#............................
....................................#.............................................#.......#................#.......###............
..##.......#......................#..#........................................................#...................#...............
.....#...........#..................................#..............#.

49